In [7]:
import numpy as np
from coins import generate_combinations
from itertools import product



In [125]:
uniform = np.asarray(list(product(range(10),range(10),range(10),range(10),range(10))))
combinations = np.asarray(generate_combinations(25, range(10), 5))
combinations_set = set(tuple(c) for c in combinations)

marginals = np.zeros(10)
for i in xrange(10):
    marginals[i] = (combinations==i).mean()
print 'Marginals', marginals
print 'Got {} combinations out of 10^5=100000'.format(len(combinations))
print 'Ratio is {}'.format(len(combinations) / 10.**5)
print 'Uniform entropy {:.2f} bits'.format(np.log(10.**5))
marginal_entropy = -5 * np.sum(marginals * np.log(marginals))
print 'Marginal entropy {:.2f} bits'.format(marginal_entropy)
joint_entropy = np.log(len(combinations))
print 'Joint entropy {:.2f} bits'.format(joint_entropy)

print 'Difference is {:.2f} bits'.format(marginal_entropy - joint_entropy)

Marginals [0.06180075 0.07369917 0.08524241 0.09589771 0.1051323  0.11241343
 0.11720831 0.11898419 0.11720831 0.11241343]
Got 5631 combinations out of 10^5=100000
Ratio is 0.05631
Uniform entropy 11.51 bits
Marginal entropy 11.41 bits
Joint entropy 8.64 bits
Difference is 2.78 bits


## Symbolic
The entropy is the lowest negative-log-likelihood one can expect (when the model matches the true distribution).
So, given symbolic-sum-25 distribution, the best nll we can expect is 8.64 bits.

Another perspective: marginal entropy is the best nll one can expect from a independent model (interestingly it's almost the same as the uniform).

N is the number of combinations (5631), i is the combination index (0 to 5630), j is the digit index (0 to 4).

NLL Joint: $$ E_p[-\log q(z)] = \frac 1 N \sum_i -\log q(z) \approx 11.41$$
NLL Marginal: $$ E_p[-\log q(z)] = E_p[-\sum_j \log q_j(z_j)] \approx 8.64$$

## Images
Now let's look into using images. We factor the model as $q(x) = q(z) \prod_i q(x_i|z_i)$. Then we need to marginalize wrt $z$ because it is an unobserved variable.

NLL Joint: $$ E_p[- \log \sum_z q(z) \prod_j q(x_j|z_j)] $$
NLL Marginal: $$ E_p[- \log \sum_z \prod_j q(z_j) q(x_j|z_j)]$$
E_p[- \log \prod_j \sum_{z_0} q(z_0) q(x_0|z_0)] = - 5 * NLL-Single-Model$$.

## Simplifying assumption of good and bad images
Let's make a simplifying assumption that:
- if $y_j=z_j$ then $q(x_j|z_j)=a$
- else if $y_j\neq z_j$ then $q(x_j|z_j)=b$.

Then we have

NLL Joint: $$ E_{y\sim p(y)}[- \log \sum_z q(z) \prod_j (a*1_{z_j=y_j} + b*1_{z_j\neq y_j})] $$

## Perfect image model
Suppose we got the "perfect" conditional image model, which is uniform over 6000 images of given class.
Then $a=1/6000$ and $b=0$.

We get a difference of 2.78 bits in that case, exactly like the symbolic case (since images are either assigned perfect score or none).

We have: 

NLL Joint: $$ E_{y\sim p(y)}[- \log (q(z) a^5)] = -5 \log a + E_{y\sim p(y)}[-\log q(z)] $$
which is 43 + Symbolic_NLL.

Indeed this result is confirmed by the numerical results with a=1/6000 and b=0

## Perfect symbolic model

NLL Joint: $$ E_{y\sim p(y)}[- \log \sum_z p(z) \prod_j (a*1_{z_j=y_j} + b*1_{z_j\neq y_j})] 
= \frac 1 N \sum_{k=1}^N [ - \log \frac 1 N \sum_{i=1}^N \prod_j (a*1_{z_j^i=y_j^k} + b*1_{z_j^i\neq y_j^k})] $$




In [81]:
# Fill perfect joint distribution
q_join = np.zeros(10**5)
for i, u_i in enumerate(uniform):
    if tuple(u_i) in combinations_set:
        q_join[i] = 1. / len(combinations_set)
print q_join.sum()

1.0


In [80]:
# Fill perfect marginal distribution
q_ind = np.zeros(10**5)
for i, u_i in enumerate(uniform):
    q_ind[i] = np.prod([marginals[u_i_j] for u_i_j in u_i])
print q_ind.sum()

1.0000000000000002


In [120]:
# Fast
def get_nll(a, b, q):
    nll = 0.
    for i, y in enumerate(combinations):
        log_arg = 0.

        log_arg = q.dot(np.prod((uniform == y) * a + (uniform != y) * b, axis=1))

        nll += -np.log(log_arg) / float(len(combinations))

        if i % 500 == 0:
            print '{}/{}'.format(i, len(combinations))
            print 'combination', y
            print 'nll', nll
    return nll

In [123]:
a = 1./6000.
b = 0.
q = q_join
nll_join = get_nll(a, b, q_join)
print nll_join

0/5631
combination [0 0 7 9 9]
nll 0.00925832286739577
500/5631
combination [1 5 9 7 3]
nll 4.638419756565316
1000/5631
combination [2 6 7 3 7]
nll 9.267581190263275
1500/5631
combination [3 6 2 7 7]
nll 13.896742623961234
2000/5631
combination [4 4 8 9 0]
nll 18.525904057659194
2500/5631
combination [5 2 8 9 1]
nll 23.155065491357153
3000/5631
combination [5 9 7 4 0]
nll 27.784226925055112
3500/5631
combination [6 7 4 3 5]
nll 32.41338835875307
4000/5631
combination [7 4 9 3 2]
nll 37.04254979245103
4500/5631
combination [8 2 5 1 9]
nll 41.671711226148986
5000/5631
combination [9 0 0 9 7]
nll 46.300872659846945
5500/5631
combination [9 7 0 5 4]
nll 50.930034093544904
52.133616066306374


In [122]:
a = 1./6000.
b = 0.
nll_ind = get_nll(a, b, q_ind)
print nll_ind

0/5631
combination [0 0 7 9 9]
nll 0.009867725421967987
500/5631
combination [1 5 9 7 3]
nll 4.901380586741098
1000/5631
combination [2 6 7 3 7]
nll 9.78055439383192
1500/5631
combination [3 6 2 7 7]
nll 14.65394538315527
2000/5631
combination [4 4 8 9 0]
nll 19.523288202264716
2500/5631
combination [5 2 8 9 1]
nll 24.39140705760259
3000/5631
combination [5 9 7 4 0]
nll 29.255284277965057
3500/5631
combination [6 7 4 3 5]
nll 34.12155291187345
4000/5631
combination [7 4 9 3 2]
nll 38.995544088353604
4500/5631
combination [8 2 5 1 9]
nll 43.87361336336889
5000/5631
combination [9 0 0 9 7]
nll 48.75141814475452
5500/5631
combination [9 7 0 5 4]
nll 53.63896196483282
54.91244098275452


In [126]:
print 'Difference is {:.2f} bits'.format(nll_ind - nll_join)

Difference is 2.78 bits


In [76]:
# Slow
q = {tuple(y): 1. / len(combinations) for y in combinations}
nll = 0.
for i, y in enumerate(combinations):
    log_arg = 0.
    print 'combination', y
    print '{}/{}'.format(i, len(combinations))
    for z in uniform:
        
        np.prod((uniform == [0, 0, 1, 0, 1]) * a + (uniform != [0, 0, 1, 0, 1]) *  , axis=1)
        
        visual_prob = 1.
        for j in xrange(5):
            if y[j] == z[j]:
                visual_prob *= a
            else:
                visual_prob *= b
        log_arg += q.get(tuple(y), 0.) * visual_prob
    nll += -np.log(log_arg)

9.015059200818782e-07

In [94]:
q * np.prod((uniform == [0, 0, 1, 0, 1]) * 3 + (uniform != [0, 0, 1, 0, 1]) * 2, axis=1)

array([9.73626394e-05, 1.74161618e-04, 1.34293296e-04, ...,
       6.08011767e-04, 5.98936965e-04, 5.74434998e-04])

In [87]:
np.equal(uniform, [0,0,1,0,1])

array([[ True,  True, False,  True, False],
       [ True,  True, False,  True,  True],
       [ True,  True, False,  True, False],
       ...,
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False]])